In [6]:
!pip install fastapi uvicorn pydantic pytz


from fastapi import FastAPI, HTTPException, Query
from pydantic import BaseModel, EmailStr
from typing import List, Optional
from datetime import datetime, timedelta
import pytz
import uuid

In [10]:
pip install pydantic[email]

Note: you may need to restart the kernel to use updated packages.


In [12]:
app = FastAPI()

# IST timezone
IST = pytz.timezone("Asia/Kolkata")

# Sample class data
class_data = [
    {
        "id": "1",
        "name": "Yoga",
        "datetime": IST.localize(datetime(2025, 7, 18, 10, 0)),
        "instructor": "Anjali",
        "available_slots": 5
    },
    {
        "id": "2",
        "name": "Zumba",
        "datetime": IST.localize(datetime(2025, 7, 18, 12, 0)),
        "instructor": "Raj",
        "available_slots": 3
    },
    {
        "id": "3",
        "name": "HIIT",
        "datetime": IST.localize(datetime(2025, 7, 19, 9, 0)),
        "instructor": "Sneha",
        "available_slots": 4
    }
]

bookings = []

class BookingRequest(BaseModel):
    class_id: str
    client_name: str
    client_email: EmailStr

class Booking(BaseModel):
    id: str
    class_id: str
    class_name: str
    datetime: str
    client_name: str
    client_email: EmailStr

@app.get("/classes")
def get_classes(timezone: Optional[str] = Query("Asia/Kolkata")):
    try:
        target_tz = pytz.timezone(timezone)
    except Exception:
        raise HTTPException(status_code=400, detail="Invalid timezone")

    converted_classes = []
    for c in class_data:
        local_time = c['datetime'].astimezone(target_tz)
        converted_classes.append({
            "id": c["id"],
            "name": c["name"],
            "datetime": local_time.strftime("%Y-%m-%d %H:%M %Z"),
            "instructor": c["instructor"],
            "available_slots": c["available_slots"]
        })
    return converted_classes

@app.post("/book")
def book_class(booking_req: BookingRequest):
    selected_class = next((c for c in class_data if c["id"] == booking_req.class_id), None)
    if not selected_class:
        raise HTTPException(status_code=404, detail="Class not found")

    if selected_class["available_slots"] <= 0:
        raise HTTPException(status_code=400, detail="No slots available")

    selected_class["available_slots"] -= 1
    booking = Booking(
        id=str(uuid.uuid4()),
        class_id=selected_class["id"],
        class_name=selected_class["name"],
        datetime=selected_class["datetime"].strftime("%Y-%m-%d %H:%M %Z"),
        client_name=booking_req.client_name,
        client_email=booking_req.client_email
    )
    bookings.append(booking)
    return {"message": "Booking successful", "booking": booking}

@app.get("/bookings")
def get_bookings(email: EmailStr):
    user_bookings = [b for b in bookings if b.client_email == email]
    return user_bookings


In [14]:
!pip install fastapi uvicorn nest_asyncio pyngrok pydantic pytz


In [16]:
from fastapi import FastAPI, HTTPException, Query
from pydantic import BaseModel, EmailStr
from typing import List, Optional
from datetime import datetime
import pytz
import uuid
import logging
import sqlite3
import os

app = FastAPI()

# Logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# IST timezone
IST = pytz.timezone("Asia/Kolkata")

# Initialize SQLite in-memory database
DB_NAME = "bookings.db"

if os.path.exists(DB_NAME):
    os.remove(DB_NAME)

conn = sqlite3.connect(DB_NAME, check_same_thread=False)
c = conn.cursor()

# Create tables
c.execute('''CREATE TABLE classes (
    id TEXT PRIMARY KEY,
    name TEXT,
    datetime TEXT,
    instructor TEXT,
    available_slots INTEGER
)''')

c.execute('''CREATE TABLE bookings (
    id TEXT PRIMARY KEY,
    class_id TEXT,
    class_name TEXT,
    datetime TEXT,
    client_name TEXT,
    client_email TEXT
)''')

# Seed data
classes_seed = [
    ("1", "Yoga", IST.localize(datetime(2025, 7, 18, 10, 0)).isoformat(), "Anjali", 5),
    ("2", "Zumba", IST.localize(datetime(2025, 7, 18, 12, 0)).isoformat(), "Raj", 3),
    ("3", "HIIT", IST.localize(datetime(2025, 7, 19, 9, 0)).isoformat(), "Sneha", 4)
]
c.executemany("INSERT INTO classes VALUES (?, ?, ?, ?, ?)", classes_seed)
conn.commit()

# Models
class BookingRequest(BaseModel):
    class_id: str
    client_name: str
    client_email: EmailStr

class Booking(BaseModel):
    id: str
    class_id: str
    class_name: str
    datetime: str
    client_name: str
    client_email: EmailStr

@app.get("/classes")
def get_classes(timezone: Optional[str] = Query("Asia/Kolkata")):
    try:
        target_tz = pytz.timezone(timezone)
    except Exception:
        raise HTTPException(status_code=400, detail="Invalid timezone")

    c.execute("SELECT * FROM classes")
    rows = c.fetchall()
    result = []
    for row in rows:
        class_time = IST.localize(datetime.fromisoformat(row[2])).astimezone(target_tz)
        result.append({
            "id": row[0],
            "name": row[1],
            "datetime": class_time.strftime("%Y-%m-%d %H:%M %Z"),
            "instructor": row[3],
            "available_slots": row[4]
        })
    return result

@app.post("/book")
def book_class(booking_req: BookingRequest):
    c.execute("SELECT * FROM classes WHERE id = ?", (booking_req.class_id,))
    class_row = c.fetchone()
    if not class_row:
        raise HTTPException(status_code=404, detail="Class not found")

    if class_row[4] <= 0:
        raise HTTPException(status_code=400, detail="No slots available")

    booking_id = str(uuid.uuid4())
    c.execute("UPDATE classes SET available_slots = available_slots - 1 WHERE id = ?", (booking_req.class_id,))
    conn.commit()

    c.execute("INSERT INTO bookings VALUES (?, ?, ?, ?, ?, ?)", (
        booking_id,
        class_row[0],
        class_row[1],
        class_row[2],
        booking_req.client_name,
        booking_req.client_email
    ))
    conn.commit()

    booking = Booking(
        id=booking_id,
        class_id=class_row[0],
        class_name=class_row[1],
        datetime=class_row[2],
        client_name=booking_req.client_name,
        client_email=booking_req.client_email
    )

    logging.info(f"Booking successful: {booking.dict()}")
    return {"message": "Booking successful", "booking": booking}

@app.get("/bookings")
def get_bookings(email: EmailStr):
    c.execute("SELECT * FROM bookings WHERE client_email = ?", (email,))
    rows = c.fetchall()
    bookings = [
        {
            "id": row[0],
            "class_id": row[1],
            "class_name": row[2],
            "datetime": row[3],
            "client_name": row[4],
            "client_email": row[5]
        } for row in rows
    ]
    return bookings
